In [2]:
import FinanceDataReader as fdr
import datetime

start_date = datetime.datetime.now() - datetime.timedelta(days=365*20)
# 현재 시간으로부터 20년 전 날짜 계산


df_kospi = fdr.DataReader('KS11', start_date).add_suffix('_kospi')
df_kosdaq = fdr.DataReader('KQ11', start_date).add_suffix('_kosdaq')
df_dow = fdr.DataReader('DJI', start_date).add_suffix('_dow')
df_nasdaq = fdr.DataReader('IXIC', start_date).add_suffix('_nasdaq')
df_snp500 = fdr.DataReader('US500', start_date).add_suffix('_s&p500')

In [4]:
import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [6]:
df_kospi

,Open_kospi,High_kospi,Low_kospi,Close_kospi,Adj Close_kospi,Volume_kospi
Date,,,,,,
2003-03-07,547.890015,555.380005,544.090027,546.020020,546.020020,635100.0
2003-03-10,545.539978,548.289978,541.289978,544.239990,544.239990,607100.0
2003-03-11,531.780029,538.619995,530.549988,532.530029,532.530029,657900.0
2003-03-12,531.169983,534.760010,524.530029,531.809998,531.809998,793800.0
2003-03-13,530.859985,531.780029,514.429993,531.780029,531.780029,679700.0
...,...,...,...,...,...,...
2023-02-22,2430.870117,2435.979980,2416.159912,2417.679932,2417.679932,426700.0
2023-02-23,2430.010010,2449.620117,2422.620117,2439.090088,2439.090088,400300.0
2023-02-24,2442.489990,2452.969971,2421.469971,2423.610107,2423.610107,369200.0


In [7]:
df_heelim = fdr.DataReader('037440', start_date).add_suffix('_heelim')
df_aps = fdr.DataReader('054620', start_date).add_suffix('_aps_holdings')
data_aps = pd.merge(df_kospi, df_kosdaq, on='Date').merge(df_dow, on='Date').merge(df_nasdaq, on='Date').merge(df_snp500, on='Date').merge(df_aps, on='Date')
data_heelim = pd.merge(df_kospi, df_kosdaq, on='Date').merge(df_dow, on='Date').merge(df_nasdaq, on='Date').merge(df_snp500, on='Date').merge(df_heelim, on='Date')